Will create data for days 0-750 then fit a forecaster on it to predict logReturns and then export the model and attempt to runin a sim without retraining.

In [72]:
import glob
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from IPython.display import display
from skforecast.plot import set_dark_theme
from tqdm import tqdm

import sklearn
import skforecast
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
from skforecast.model_selection import (
    TimeSeriesFold,
    OneStepAheadFold,
    backtesting_forecaster,
    bayesian_search_forecaster,
    backtesting_forecaster_multiseries,
    bayesian_search_forecaster_multiseries
)
from skforecast.preprocessing import RollingFeatures, series_long_to_dict, exog_long_to_dict
from skforecast.exceptions import OneStepAheadValidationWarning

import warnings

In [85]:
prices1000Days = np.loadtxt(project_root / "sourceCode" / "1000Prices.txt").T

from strategies.ms_forecasting.main import createGreeksManager
from strategies.ms_forecasting.main import GreeksManager

trainingDataWindowSize = 730 # 20 days of data needed for all greeks to be void of NaN's
gm: GreeksManager = createGreeksManager(prices1000Days[:, :751], trainingDataWindowSize) # giving 751 days but the 751'st is removed when we get the history

exogIndex = pd.RangeIndex(20, 750)
exogDict = gm.getGreeksHistoryDict(exogIndex)

print("Inst 0 days [742, 750]:\n", prices1000Days[0, 742:750])
print("inst 0 head:\n", exogDict["inst_0"].tail(1))
print(f"each inst-greek has {len(exogDict["inst_0"]["greek_lag_1"])} days")

Inst 0 days [742, 750]:
 [36.81 36.75 36.53 36.49 36.3  36.58 36.43 36.24]
inst 0 head:
      greek_lag_1  greek_lag_2  greek_lag_3  greek_lag_4  greek_lag_5  \
749        36.43        36.58         36.3        36.49        36.53   

     greek_lag_6  greek_lag_7  greek_volatility_5  greek_volatility_10  \
749        36.75        36.81            0.005455             0.004396   

     greek_volatility_20  greek_momentum_3  greek_momentum_7  \
749              0.00372         -0.001654         -0.015606   

     greek_momentum_14  greek_price  
749          -0.028562        36.24  
each inst-greek has 730 days


In [86]:
logReturnsPricesWindow = prices1000Days[:, 20:]
logReturnsNp = np.log(logReturnsPricesWindow[:, 1:] / logReturnsPricesWindow[:, :-1])

logReturnsIndex = pd.RangeIndex(20, 999)
logReturnsSeries = pd.DataFrame(logReturnsNp.T,
                          index = logReturnsIndex,
                          columns = [f"inst_{i}" for i in range(logReturnsNp.shape[0])])

logReturnsTrainingSeries = logReturnsSeries.iloc[:trainingDataWindowSize]

In [87]:
logReturnsForecaster = ForecasterRecursiveMultiSeries(
    regressor           = HistGradientBoostingRegressor(random_state=8523,
                                                        learning_rate=0.05),
    transformer_series  = None,
    transformer_exog    = StandardScaler(),
    lags                = 20,
    window_features     = RollingFeatures(
                                stats           = ['min', 'max', 'ewm'],
                                window_sizes    = 10,
                            ),
)

logReturnsForecaster.fit(
    series  = logReturnsTrainingSeries,
    exog    = exogDict
)

logReturnsForecaster.is_fitted

True

In [ ]:
predictions = pd.DataFrame(
    index=pd.RangeIndex(750, 1000),
    columns=logReturnsSeries.columns  # inst_0…inst_49
)

n_lags = max(logReturnsForecaster.lags)  # 20 in your case

for day in range(750, 1000):
    gm.updateGreeks(prices1000Days[:, day])

    exog = gm.getGreeksDict(pd.RangeIndex(day, day+1))

    last_window = logReturnsSeries.iloc[-n_lags:]

    pred = logReturnsForecaster.predict(
        steps       = 1,
        last_window = last_window,
        exog        = exog
    )
    predictions.loc[day] = pred["pred"].values

    # 5) now append the true return for day so next iteration's last_window grows
    true_ret = np.log(prices1000Days[:, day] / prices1000Days[:, day-1])
    logReturnsSeries.loc[day] = true_ret


╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ To make predictions `exog` for series 'inst_0' must start one step ahead of          │
│ `last_window`. Missing values are filled with NaN.                                   │
│     `last_window` ends at : 998.                                                     │
│     `exog` for series 'inst_0' starts at : 750.                                      │
│      Expected index       : 999.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ C:\Users\jense\anaconda3\envs\Algothon-2025\Lib\site-packages\skforecast\utils\utils │
│ .py:1134                                                                             │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ To make predictions `exog` for series 'inst_1' must start one step ahead of          │
│ `last_window`. Missing values are filled with NaN.                                   │
│     `last_window` ends at : 998.                                                     │
│     `exog` for series 'inst_1' starts at : 750.                                      │
│      Expected index       : 999.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ C:\Users\jense\anaconda3\envs\Algothon-2025\Lib\site-packages\skforecast\utils\utils │
│ .py:1134                                                                             │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ To make predictions `exog` for series 'inst_2' must start one step ahead of          │
│ `last_window`. Missing values are filled with NaN.                                   │
│     `last_window` ends at : 998.                                                     │
│     `exog` for series 'inst_2' starts at : 750.                                      │
│      Expected index       : 999.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ C:\Users\jense\anaconda3\envs\Algothon-2025\Lib\site-packages\skforecast\utils\utils │
│ .py:1134                                                                             │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ To make predictions `exog` for series 'inst_3' must start one step ahead of          │
│ `last_window`. Missing values are filled with NaN.                                   │
│     `last_window` ends at : 998.                                                     │
│     `exog` for series 'inst_3' starts at : 750.                                      │
│      Expected index       : 999.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ C:\Users\jense\anaconda3\envs\Algothon-2025\Lib\site-packages\skforecast\utils\utils │
│ .py:1134                                                                             │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ To make predictions `exog` for series 'inst_4' must start one step ahead of          │
│ `last_window`. Missing values are filled with NaN.                                   │
│     `last_window` ends at : 998.                                                     │
│     `exog` for series 'inst_4' starts at : 750.                                      │
│      Expected index       : 999.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ C:\Users\jense\anaconda3\envs\Algothon-2025\Lib\site-packages\skforecast\utils\utils │
│ .py:1134                                                                             │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ To make predictions `exog` for series 'inst_15' must start one step ahead of         │
│ `last_window`. Missing values are filled with NaN.                                   │
│     `last_window` ends at : 998.                                                     │
│     `exog` for series 'inst_15' starts at : 756.                                     │
│      Expected index       : 999.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ C:\Users\jense\anaconda3\envs\Algothon-2025\Lib\site-packages\skforecast\utils\utils │
│ .py:1134                                                                             │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ To make predictions `exog` for series 'inst_16' must start one step ahead of         │
│ `last_window`. Missing values are filled with NaN.                                   │
│     `last_window` ends at : 998.                                                     │
│     `exog` for series 'inst_16' starts at : 756.                                     │
│      Expected index       : 999.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ C:\Users\jense\anaconda3\envs\Algothon-2025\Lib\site-packages\skforecast\utils\utils │
│ .py:1134                                                                             │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ To make predictions `exog` for series 'inst_17' must start one step ahead of         │
│ `last_window`. Missing values are filled with NaN.                                   │
│     `last_window` ends at : 998.                                                     │
│     `exog` for series 'inst_17' starts at : 756.                                     │
│      Expected index       : 999.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ C:\Users\jense\anaconda3\envs\Algothon-2025\Lib\site-packages\skforecast\utils\utils │
│ .py:1134                                                                             │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ To make predictions `exog` for series 'inst_18' must start one step ahead of         │
│ `last_window`. Missing values are filled with NaN.                                   │
│     `last_window` ends at : 998.                                                     │
│     `exog` for series 'inst_18' starts at : 756.                                     │
│      Expected index       : 999.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ C:\Users\jense\anaconda3\envs\Algothon-2025\Lib\site-packages\skforecast\utils\utils │
│ .py:1134                                                                             │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────── MissingValuesWarning ────────────────────────────────╮
│ To make predictions `exog` for series 'inst_19' must start one step ahead of         │
│ `last_window`. Missing values are filled with NaN.                                   │
│     `last_window` ends at : 998.                                                     │
│     `exog` for series 'inst_19' starts at : 756.                                     │
│      Expected index       : 999.                                                     │
│                                                                                      │
│ Category : MissingValuesWarning                                                      │
│ Location :                                                                           │
│ C:\Users\jense\anaconda3\envs\Algothon-2025\Lib\site-packages\skforecast\utils\utils │
│ .py:1134                                                                             │
│ Suppress : warnings.simplefilter('ignore', category=MissingValuesWarning)            │
╰──────────────────────────────────────────────────────────────────────────────────────╯